In [1]:
%pip install -q prince

In [2]:
import prince

dataset = prince.datasets.load_energy_mix(year=2019, normalize=True)
dataset.head().style.format('{:.0%}')

,,coal,oil,gas,nuclear,hydro,wind,solar,other renewables
continent,country,,,,,,,,
Africa,Algeria,1%,35%,64%,0%,0%,0%,0%,0%
South America,Argentina,1%,35%,50%,2%,10%,1%,0%,1%
Oceania,Australia,28%,34%,30%,0%,2%,3%,3%,1%
Europe,Austria,9%,37%,22%,0%,25%,4%,1%,3%
Asia,Azerbaijan,0%,33%,65%,0%,2%,0%,0%,0%


In [5]:
pca = prince.PCA(
    n_components=3,
    n_iter=3,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=42
)
pca = pca.fit(
    dataset,
    sample_weight=None,
    column_weight=None,
    supplementary_columns=None
)
pca.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,1.963,24.54%,24.54%
1,1.561,19.51%,44.05%
2,1.403,17.54%,61.59%


In [8]:
pca.eigenvalues_

array([1.96301214, 1.56069986, 1.40327823])

In [9]:
pca.percentage_of_variance_

array([24.5376518 , 19.50874822, 17.54097785])

In [10]:
pca.cumulative_percentage_of_variance_

array([24.5376518 , 44.04640003, 61.58737788])

In [12]:
pca.scree_plot()

alt.Chart(...)

In [13]:
pca.transform(dataset).head()

,component,0,1,2
continent,country,,,
Africa,Algeria,-2.189068,0.380243,-0.388572
South America,Argentina,-1.244981,0.801917,-0.389456
Oceania,Australia,0.203969,-1.470254,0.531819
Europe,Austria,0.847122,1.008296,-0.521998
Asia,Azerbaijan,-2.190535,0.632250,-0.365515


In [14]:
pca.transform(dataset).equals(pca.row_coordinates(dataset))

True

In [15]:
pca.column_coordinates_

component,0,1,2
variable,,,
coal,0.230205,-0.395474,0.740748
oil,0.176200,-0.418917,-0.737855
gas,-0.866927,0.182990,-0.096857
nuclear,0.310313,-0.002598,0.400608
hydro,0.440383,0.744815,-0.016375
wind,0.518712,-0.161507,-0.364593
solar,0.415677,-0.589288,0.001012
other renewables,0.628750,0.516935,-0.084114


In [16]:
pca.plot(
    dataset,
    x_component=0,
    y_component=1,
    color_rows_by='continent',
    show_row_markers=True,
    show_column_markers=True,
    show_row_labels=False,
    row_labels_column=None,  # for DataFrames with a MultiIndex
    show_column_labels=False
)

alt.LayerChart(...)

In [17]:
pca.row_contributions_.head().style.format('{:.0%}')

,component,0,1,2
continent,country,,,
Africa,Algeria,3%,0%,0%
South America,Argentina,1%,1%,0%
Oceania,Australia,0%,2%,0%
Europe,Austria,0%,1%,0%
Asia,Azerbaijan,3%,0%,0%


In [18]:
pca.column_contributions_.style.format('{:.0%}')

component,0,1,2
variable,,,
coal,3%,10%,39%
oil,2%,11%,39%
gas,38%,2%,1%
nuclear,5%,0%,11%
hydro,10%,36%,0%
wind,14%,2%,9%
solar,9%,22%,0%
other renewables,20%,17%,1%


In [19]:
pca.row_cosine_similarities(dataset).head()

,component,0,1,2
continent,country,,,
Africa,Algeria,0.902764,0.027238,0.028445
South America,Argentina,0.626517,0.259936,0.061309
Oceania,Australia,0.008468,0.439999,0.057570
Europe,Austria,0.236087,0.334470,0.089643
Asia,Azerbaijan,0.868801,0.072377,0.024190


In [20]:
pca.column_cosine_similarities_

component,0,1,2
variable,,,
coal,0.052994,0.156400,0.548707
oil,0.031046,0.175492,0.544430
gas,0.751563,0.033485,0.009381
nuclear,0.096294,0.000007,0.160487
hydro,0.193937,0.554750,0.000268
wind,0.269062,0.026085,0.132928
solar,0.172788,0.347260,0.000001
other renewables,0.395327,0.267222,0.007075


In [21]:
pca.column_correlations

component,0,1,2
variable,,,
coal,0.230205,-0.395474,0.740748
oil,0.176200,-0.418917,-0.737855
gas,-0.866927,0.182990,-0.096857
nuclear,0.310313,-0.002598,0.400608
hydro,0.440383,0.744815,-0.016375
wind,0.518712,-0.161507,-0.364593
solar,0.415677,-0.589288,0.001012
other renewables,0.628750,0.516935,-0.084114


In [22]:
(pca.column_correlations ** 2).equals(pca.column_cosine_similarities_)

True

In [23]:
reconstructed = pca.inverse_transform(pca.transform(dataset))
reconstructed.head()

,,0,1,2,3,4,5,6,7
continent,country,,,,,,,,
Africa,Algeria,0.024204,0.369866,0.599244,-0.007082,0.024826,-0.003249,0.000650,-0.008460
South America,Argentina,0.027773,0.366342,0.487562,0.008596,0.090082,0.006963,0.001391,0.011292
Oceania,Australia,0.287372,0.425348,0.208594,0.056641,-0.018484,0.025867,0.015104,-0.000441
Europe,Austria,0.060875,0.410393,0.219553,0.040122,0.183231,0.034237,0.006167,0.045423
Asia,Azerbaijan,0.013098,0.354083,0.606928,-0.006558,0.042620,-0.004640,-0.000439,-0.005092


In [24]:
import numpy as np

np.linalg.norm(reconstructed.values - dataset.values, ord=2)

np.float64(1.192715958987243)

In [26]:
active = dataset.query("continent != 'North America'")
pca = prince.PCA().fit(active)
pca.transform(dataset).query("continent == 'North America'")

component                                 0         1
continent     country                                
North America Canada               0.009864  1.332859
              Mexico              -0.695023 -0.402230
              Trinidad and Tobago -3.072833  1.393067
              United States       -0.226122 -0.433260

In [27]:
pca = prince.PCA().fit(dataset, supplementary_columns=['hydro', 'wind', 'solar'])
pca.column_correlations

component,0,1
variable,,
coal,0.621633,-0.380008
oil,0.048966,0.948650
gas,-0.916078,-0.331546
nuclear,0.384458,-0.401675
other renewables,0.467116,0.086656
hydro,2.192240,0.329911
wind,1.739201,1.637625
solar,2.233130,0.677610


In [28]:
pca = prince.PCA().fit(active, supplementary_columns=['hydro', 'wind', 'solar'])

In [29]:
row_weights = np.ones(active.shape[0])
row_weights[active.index.get_level_values('continent') == 'Europe'] = 2

column_weights = np.ones(active.shape[1])
column_weights[active.columns.isin(['hydro', 'wind', 'solar', 'other renewables'])] = 2

pca = prince.PCA().fit(
    active,
    sample_weight=row_weights,
    column_weight=column_weights
)
pca.transform(dataset).head()

,component,0,1
continent,country,,
Africa,Algeria,-1.512601,-1.806884
South America,Argentina,-0.403372,-1.475517
Oceania,Australia,-1.441339,1.394480
Europe,Austria,1.521697,0.040025
Asia,Azerbaijan,-1.272744,-1.980454
